# Comparison with classical models

In this notebook we provide a comparison between the hybrid network and some well-known classical CNNs. 
The considered classical models are:

1. ResNet50
2. DenseNet169
3. VGG-19
4. EfficientNet
5. Inception



Import all the necessary libraries

In [ ]:
import os
import numpy as np
import torch
import torch.utils.data as data
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
from torchvision import datasets, models, transforms

import glob
import random
import numpy as np
import argparse

from PIL import Image
from google.colab import drive
import zipfile

np.random.seed = 12
torch.manual_seed(12)
random.seed(12)

Mounting data-set files from Google Drive. 

For more information see the instruction in [QuantumProjectBinucci.ipynb](https://colab.research.google.com/drive/1Yoh6_thHPRiti2YuuZqGIBGLyr2UKjqP?usp=sharing)

In [ ]:
drive.mount('/content/gdrive')
zip_ref = zipfile.ZipFile("/content/gdrive/MyDrive/Progetto Quantum Binucci PhD/Data/archive.zip", 'r')
zip_ref.extractall("/content/dataset")
zip_ref.close()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Dataset** class, used in order to manage the dataset loading. 

In [ ]:
class Dataset(data.Dataset):
    # mapping table of label and index


    def __init__(self, train, **kwargs):
        super(Dataset, self).__init__()

        self.str2label = {"buildings": 0, "forest": 1, "glacier": 2, "mountain": 3, "sea": 4, "street": 5}
        self.label2str = {0: "buildings", 1: "forest", 2: "glacier", 3: "mountain", 4: "sea", 5: "street"}

        self.data = list()
        self.size = kwargs.get("size", None)
        self.data_root = kwargs.get("data_root", "./Data")
        # self.data_root = os.path.join(os.path.dirname(os.path.realpath(__file__)), 'data')

        # load csv file and import file paths
        main_dir = "seg_train" if train else "seg_test"
        print(os.path.join(self.data_root, main_dir))

        for current_dir in os.listdir(os.path.join(self.data_root,main_dir)):
            for current_file in os.listdir(os.path.join(self.data_root,main_dir,current_dir)):
                path = os.path.join(self.data_root,main_dir,current_dir,current_file)
                self.data.append((path,current_dir))

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __getitem__(self, index):
        path, label = self.data[index]
        image = Image.open(path)

        # resize input images
        if self.size:
            image = image.resize((self.size, self.size), Image.BICUBIC)

        label = self.str2label[label]

        return self.transform(image), label

    def __len__(self):
        return len(self.data)

**Solver** class, used in order to manage the train/evaluation process

In [ ]:
class Solver():
    def __init__(self, args,**kwargs):
        # prepare a dataset
        self.train_data = Dataset(train=True,
                                  data_root=args.data_root,
                                  size=args.image_size)


        self.test_data = Dataset(train=False,
                                 data_root=args.data_root,
                                 size=args.image_size)
        
        #Splitting the data-set in training/validation. For reproducibility purposes we consider fixed
        #seed (42).
        lengths = [int(0.2*len(self.train_data)), len(self.train_data) - int(0.2 * len(self.train_data))]
        self.train_set, self.val_set = torch.utils.data.random_split(self.train_data,
                                                           lengths,
                                                           torch.Generator().manual_seed(42))

        self.train_loader = DataLoader(dataset=self.train_set,
                                       batch_size=args.batch_size,
                                       shuffle=True, drop_last=True)
        
        self.test_loader = DataLoader(dataset=self.test_data,
                                       batch_size=args.batch_size,
                                       shuffle=True, drop_last=True)

        # turn on the CUDA if available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        print(f'Current device: {self.device}')
        ########################################################################
        #Loading network to train/test. By default, the script load the pre-trained ResNet50
        self.net = kwargs.get('net',models.resnet50(pretrained=True)).to(self.device)
        print(f'You requested the training of: {self.net.__class__.__name__}')
        ########################################################################


        self.loss_fn = torch.nn.CrossEntropyLoss()
        self.optim = torch.optim.Adam(self.net.parameters(), lr=args.lr)
        self.args = args

        if not os.path.exists(args.ckpt_dir):
            os.makedirs(args.ckpt_dir)

    def fit(self):
        max_accuracy = 0
      
        args = self.args
        for epoch in range(args.max_epochs):
            self.net.train()
            for step, inputs in enumerate(self.train_loader):

                
                images = inputs[0].to(self.device)
                labels = inputs[1].to(self.device)
                pred = self.net(images)
                loss = self.loss_fn(pred, labels)

                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
                if (step+1) % args.print_every_minibatches==0:
                  print("Epoch [{}/{}] Batch [{}/{}] Loss: {:.3f}".
                      format(epoch + 1, args.max_epochs,step+1, len(self.train_loader), loss.item()))
            if (epoch + 1) % args.print_every == 0:
                train_acc = self.evaluate(self.train_data)
                test_acc = self.evaluate(self.val_set)

                print("Epoch [{}/{}] Loss: {:.3f} Train Acc: {:.3f}, Test Acc: {:.3f}".
                      format(epoch + 1, args.max_epochs, loss.item(), train_acc, test_acc))
                if test_acc > max_accuracy:
                  max_accuracy = test_acc
                  self.save(args.ckpt_dir, args.ckpt_name, 'best')

    def evaluate(self, data):
        args = self.args
        loader = DataLoader(data,
                            batch_size=args.batch_size,
                            shuffle=False)

        self.net.eval()
        num_correct, num_total = 0, 0

        with torch.no_grad():
            for inputs in loader:
                images = inputs[0].to(self.device)
                labels = inputs[1].to(self.device)

                outputs = self.net(images)
                _, preds = torch.max(outputs.detach(), 1)

                num_correct += (preds == labels).sum().item()
                num_total += labels.size(0)

        return num_correct / num_total

    def test(self):
      return self.evaluate(self.test_data)

    def load_network(self,net):
      self.net = net



    def save(self, ckpt_dir, ckpt_name, global_step):
        save_path = os.path.join(
            ckpt_dir, "{}_{}.pth".format(ckpt_name, global_step))
        torch.save(self.net.state_dict(), save_path)

In [ ]:
def prepare_network_for_t_learning(net,**kwargs):
    freeze_parameters = kwargs.get('freeze_parameters',True)
    in_j_layer = kwargs.get('in_j_layer',4)
    out_j_layer = kwargs.get('out_j_layer',3)
    if freeze_parameters == True:
      for param in net.parameters():
          param.requires_grad = False
    net_name = net.__class__.__name__
    if net_name=='DenseNet':
      #Getting the number of output classes
      output_classes = kwargs.get('output_classes',6)
      num_ftrs = net.classifier.in_features

      junc_layer_1 = nn.Linear(num_ftrs,in_j_layer)
      fc_intermediate_layer = nn.Linear(in_j_layer,out_j_layer)
      junc_layer_2 = nn.Linear(out_j_layer,output_classes)

      #Adapating the last layer of the network to the specific classification task
      net.classifier = nn.Sequential(junc_layer_1,fc_intermediate_layer,junc_layer_2)
    elif net_name == 'ResNet':
      #Getting the number of output classes
      output_classes = kwargs.get('output_classes',6)
      num_ftrs = net.fc.in_features

      junc_layer_1 = nn.Linear(num_ftrs,in_j_layer)
      fc_intermediate_layer = nn.Linear(in_j_layer,out_j_layer)
      junc_layer_2 = nn.Linear(out_j_layer,output_classes)

      #Adapating the last layer of the network to the specific classification task
      net.fc = nn.Sequential(junc_layer_1,fc_intermediate_layer,junc_layer_2)
    elif net_name == 'VGG':
      output_classes = kwargs.get('output_classes',6)
      num_ftrs = net.classifier[6].in_features

      junc_layer_1 = nn.Linear(num_ftrs,in_j_layer)
      fc_intermediate_layer = nn.Linear(in_j_layer,out_j_layer)
      junc_layer_2 = nn.Linear(out_j_layer,output_classes)

      net.classifier[6] = nn.Sequential(junc_layer_1,fc_intermediate_layer,junc_layer_2)
    elif net_name == 'EfficientNet':
      output_classes = kwargs.get('output_classes',6)
      num_ftrs = net.classifier.fc.in_features
      #Adapating the last layer of the network to the specific classification task

      junc_layer_1 = nn.Linear(num_ftrs,in_j_layer)
      fc_intermediate_layer = nn.Linear(in_j_layer,out_j_layer)
      junc_layer_2 = nn.Linear(out_j_layer,output_classes)

      net.classifier.fc = nn.Sequential(junc_layer_1,fc_intermediate_layer,junc_layer_2)
    elif net_name == "Inception3":
      output_classes = kwargs.get('output_classes',6)
      num_ftrs = net.fc.in_features
      #Adapating the last layer of the network to the specific classification task

      junc_layer_1 = nn.Linear(num_ftrs,in_j_layer)
      fc_intermediate_layer = nn.Linear(in_j_layer,out_j_layer)
      junc_layer_2 = nn.Linear(out_j_layer,output_classes)

      net.fc = nn.Sequential(junc_layer_1,fc_intermediate_layer,junc_layer_2)
      net.aux_logits=False
    num_params = 0
    num_params += sum(param.numel() for param in net.parameters() if param.requires_grad)
    print('You requested the train of: ',net_name)
    print('Number of parameters:',num_params)
    return net

Train class, used in order to perform the training process.

In [ ]:
def main():
    parser = argparse.ArgumentParser()
    
    model_set = {'ResNet50': models.resnet50(pretrained=True), 
                 'DenseNet169': models.densenet169(pretrained=True), 
                 'VGG19': models.vgg19(pretrained=True),
                 'EfficientNet': torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0',
                                                pretrained=True),
                 'Inception': torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)}

    parser.add_argument("--model_to_test",type=str,default='Inception')
    parser.add_argument("--test",type=bool,default=False)
    parser.add_argument("--lr", type=float, default=1e-3)
    parser.add_argument("--batch_size", type=int, default=64)
    parser.add_argument("--max_epochs", type=int, default=30)

    parser.add_argument("--ckpt_dir", type=str, default="/content/gdrive/MyDrive/Progetto Quantum Binucci PhD/CheckpointSmallParametersNumber/VGG19/seed12")
    parser.add_argument("--path_to_test",type=str,default="/content/gdrive/MyDrive/Progetto Quantum Binucci PhD/CheckpointSmallParametersNumber/VGG19/seed12/landscape_best.pth")
    parser.add_argument("--ckpt_name", type=str, default="landscape")
    parser.add_argument("--print_every", type=int, default=1)
    parser.add_argument("--print_every_minibatches", type=int, default=10)

    # if you change image size, you must change all the network channels
    parser.add_argument("--image_size", type=int, default=299)
    parser.add_argument("--data_root", type=str, default="/content/dataset")



    args, unknown = parser.parse_known_args()

    net = prepare_network_for_t_learning(model_set[args.model_to_test],freeze_parameters = True)

    print("Test mode" if args.test else "Train Mode")

    solver = Solver(args,net=net)
    
    if args.test==False:
      solver.fit()
    else:
      net.load_state_dict(torch.load(args.path_to_test,map_location=torch.device('cpu')))
      solver.load_network(net)
      accuracy = solver.test()
      print(f'Accuracy on the test-set for {net.__class__.__name__} = {accuracy*100}%')
      
if __name__ == "__main__":
    main()

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


You requested the train of:  VGG
Number of parameters: 16427
Train Mode
/content/dataset/seg_train
/content/dataset/seg_test
Current device: cuda
You requested the training of: VGG
Epoch [1/30] Batch [10/43] Loss: 1.475
Epoch [1/30] Batch [20/43] Loss: 1.267
Epoch [1/30] Batch [30/43] Loss: 1.298
Epoch [1/30] Batch [40/43] Loss: 1.142
Epoch [1/30] Loss: 1.040 Train Acc: 0.526, Test Acc: 0.527
Epoch [2/30] Batch [10/43] Loss: 0.894
Epoch [2/30] Batch [20/43] Loss: 0.872
Epoch [2/30] Batch [30/43] Loss: 0.916
Epoch [2/30] Batch [40/43] Loss: 0.889
Epoch [2/30] Loss: 0.705 Train Acc: 0.713, Test Acc: 0.715
Epoch [3/30] Batch [10/43] Loss: 0.719
Epoch [3/30] Batch [20/43] Loss: 0.719
Epoch [3/30] Batch [30/43] Loss: 0.553
Epoch [3/30] Batch [40/43] Loss: 0.785
Epoch [3/30] Loss: 0.652 Train Acc: 0.717, Test Acc: 0.721
Epoch [4/30] Batch [10/43] Loss: 0.609
Epoch [4/30] Batch [20/43] Loss: 0.585
Epoch [4/30] Batch [30/43] Loss: 0.636
Epoch [4/30] Batch [40/43] Loss: 0.730
Epoch [4/30] Loss: